In [ ]:
# Extraindo dados da API

import requests

# definindo função que faz solicitação de dados para uma API de taxas de câmbio
def extraindo_dados(date, base="BRL"):

    #  construindo URL da API concatenando a data e a moeda base fornecidas como parâmetros
    url = f"https://api.apilayer.com/exchangerates_data/{date}?base={base}"

    # cabeçalhos da solicitação HTTP, incluindo a chave de API.
    headers= {
    "apikey": "inserir códio"
    }

    # parâmetros da solicitação, incluindo a moeda base e os símbolos das moedas desejadas para retorno.
    parametros = {
        "base": base,
        "symbols": "USD, GBP, EUR, JPY, CNY, CAD, ZAR, ARS " # Escolhendo as moedas de retorno
        }

    # envia solicitação HTTP GET para a URL da API, passando os cabeçalhos e os parâmetros definidos
    response = requests.request("GET", url, headers=headers, params=parametros)

    # verifica se a resposta da solicitação tem um código de status 200 (indicando uma resposta bem-sucedida)
    if response.status_code != 200:
        raise Exception("Não consegui extrair dados!!!") # se houver erro, exceção é levantada com uma mensagem

    # retorna os dados da resposta da API no formato JSON
    return response.json()

In [ ]:
# chamando função solicitando dados de taxas de câmbio para data de interesse
extraindo_dados("2023-01-05")

{'success': True,
 'timestamp': 1672963199,
 'historical': True,
 'base': 'BRL',
 'date': '2023-01-05',
 'rates': {'USD': 0.18687,
  'GBP': 0.156891,
  'EUR': 0.177601,
  'JPY': 24.911685,
  'CNY': 1.286136,
  'CAD': 0.253421,
  'ZAR': 3.207799,
  'ARS': 33.432882}}

In [ ]:
# retorna a data para a qual foram solicitadas as taxas de câmbio
extraindo_dados("2023-01-05")['date']

'2023-01-05'

In [ ]:
# extraindo o ano, mês e dia da data retornada pela função
ano, mes, dia = extraindo_dados("2023-01-05")['date'].split('-')
print(ano, mes, dia)

2023 01 05


In [ ]:
# lista e exibe visualmente os arquivos no diretório raiz do DBFS
display(dbutils.fs.ls('dbfs:/'))

path,name,size,modificationTime
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/volume/,volume/,0,0
dbfs:/volumes/,volumes/,0,0


In [ ]:
# criando uma estrutura de diretórios baseadas em data e no contexto de pipeline de dados
caminho_completo = f'dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}'
print(caminho_completo)

dbfs:/databricks-results/bronze/2023/01/05


In [ ]:
# função converte os dados de taxas de câmbio no formato JSON para uma lista de tuplas
def dados_para_dataframe(dado_json):
    dados_tupla = [(moeda, float (taxa)) for moeda, taxa in dado_json['rates'].items()]
    return dados_tupla

In [ ]:
# executando duas funções para extrair os dados da taxa de câmbio e convertendo para lista de tuplas
dados_para_dataframe(extraindo_dados("2023-01-05"))

[('USD', 0.18687),
 ('GBP', 0.156891),
 ('EUR', 0.177601),
 ('JPY', 24.911685),
 ('CNY', 1.286136),
 ('CAD', 0.253421),
 ('ZAR', 3.207799),
 ('ARS', 33.432882)]

In [ ]:
# guardando resultado na variável
response = dados_para_dataframe(extraindo_dados("2022-09-07"))
response

[('USD', 0.190542),
 ('GBP', 0.165411),
 ('EUR', 0.19063),
 ('JPY', 27.462719),
 ('CNY', 1.32737),
 ('CAD', 0.250192),
 ('ZAR', 3.291229),
 ('ARS', 26.819519)]

In [ ]:
# Transformando dados em DataFrame Spark
df_conversoes = spark.createDataFrame(response, schema=['moeda', 'taxa'])
df_conversoes.show()

+-----+---------+
|moeda|     taxa|
+-----+---------+
|  USD| 0.190542|
|  GBP| 0.165411|
|  EUR|  0.19063|
|  JPY|27.462719|
|  CNY|  1.32737|
|  CAD| 0.250192|
|  ZAR| 3.291229|
|  ARS|26.819519|
+-----+---------+



In [ ]:
# adicionando nova coluna ao dataframe preenchendo com valor literal com a função 'lit'
from pyspark.sql.functions import lit

df_conversoes = df_conversoes.withColumn("data", lit(f"{ano}-{mes}-{dia}"))
df_conversoes.show()

+-----+---------+----------+
|moeda|     taxa|      data|
+-----+---------+----------+
|  USD| 0.190542|2023-01-05|
|  GBP| 0.165411|2023-01-05|
|  EUR|  0.19063|2023-01-05|
|  JPY|27.462719|2023-01-05|
|  CNY|  1.32737|2023-01-05|
|  CAD| 0.250192|2023-01-05|
|  ZAR| 3.291229|2023-01-05|
|  ARS|26.819519|2023-01-05|
+-----+---------+----------+



In [ ]:
# salvando dataframe no formato Parquet utilizando o modo 'overwrite'
df_conversoes.write.format("parquet")\
    .mode("overwrite")\
    .save(caminho_completo)

In [ ]:
# exibindo conteúdo presente no endereço indicado
display(dbutils.fs.ls(f"{caminho_completo}"))

path,name,size,modificationTime
dbfs:/databricks-results/bronze/2023/01/05/_SUCCESS,_SUCCESS,0,1711723960000
dbfs:/databricks-results/bronze/2023/01/05/_committed_597107826751072451,_committed_597107826751072451,416,1711723960000
dbfs:/databricks-results/bronze/2023/01/05/_started_597107826751072451,_started_597107826751072451,0,1711723959000
dbfs:/databricks-results/bronze/2023/01/05/part-00000-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-68-1-c000.snappy.parquet,part-00000-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-68-1-c000.snappy.parquet,1039,1711723960000
dbfs:/databricks-results/bronze/2023/01/05/part-00001-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-69-1-c000.snappy.parquet,part-00001-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-69-1-c000.snappy.parquet,1038,1711723960000
dbfs:/databricks-results/bronze/2023/01/05/part-00002-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-70-1-c000.snappy.parquet,part-00002-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-70-1-c000.snappy.parquet,1039,1711723959000
dbfs:/databricks-results/bronze/2023/01/05/part-00003-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-71-1-c000.snappy.parquet,part-00003-tid-597107826751072451-66252285-d79e-49c9-990f-de1cab856d0c-71-1-c000.snappy.parquet,1039,1711723959000
